In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
from cs285.envs.pendulum.pendulum_env import PendulumEnv
from cs285.envs.dt_sampler import ConstantSampler
from cs285.infrastructure.replay_buffer import ReplayBufferTrajectories
from cs285.infrastructure.utils import sample_n_trajectories, RandomPolicy
from cs285.agents.ode_agent import ODEAgent
from typing import Callable, Optional, Tuple, Sequence
import numpy as np
import torch.nn as nn
import torch
import gym
from cs285.infrastructure import pytorch_util as ptu
from torchdiffeq import odeint
from tqdm import trange
import jax
import jax.numpy as jnp
import equinox as eqx
import diffrax
from diffrax import diffeqsolve, Dopri5
import optax

In [3]:
key = jax.random.PRNGKey(0)
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)
device = "cuda" if torch.cuda.is_available() else "cpu"

cpu


In [4]:
dt_sampler = ConstantSampler(dt=0.05)
env = PendulumEnv(
    dt_sampler=dt_sampler
)
agent_key, key = jax.random.split(key)
mb_agent = ODEAgent(
    env=env,
    hidden_size=128,
    num_layers=4,
    ensemble_size=10,
    train_timestep=0.005,
    train_discount=1,
    mpc_horizon_steps=100,
    mpc_timestep=0.005,
    mpc_strategy="cem",
    mpc_discount=0.9,
    mpc_num_action_sequences=1000,
    cem_num_iters=4,
    cem_num_elites=5,
    cem_alpha=1,
    key=agent_key
)
replay_buffer = ReplayBufferTrajectories(seed=42)
trajs, _ = sample_n_trajectories(env, RandomPolicy(env=env), ntraj=10, max_length=200, key=key)
replay_buffer.add_rollouts(trajs)

100%|██████████| 10/10 [00:00<00:00, 30.86it/s]


In [5]:
for n in trange(2):
    for i in range(mb_agent.ensemble_size):
        traj = replay_buffer.sample_rollout()
        loss = mb_agent.update(i, jnp.array(traj["observations"]), jnp.array(traj["actions"]), jnp.cumsum(traj["dts"]), discount=0.9)
# TODO: change the implementation to allow sampling batched rollouts

  0%|          | 0/2 [00:00<?, ?it/s]


TypeError: 'DeprecationWarning' object is not callable

In [14]:
loss

0.2549184560775757

In [6]:
batch_size = 64
for n in trange(2):
    for i in range(mb_agent.ensemble_size):
        obs, acs, times = jnp.empty(shape=(batch_size, 201, 3)), jnp.empty(shape=(batch_size, 201, 1)), jnp.empty(shape=(batch_size, 201))
        for m in range(batch_size):
            traj = replay_buffer.sample_rollout()
            obs.at[m].set(traj["observations"])
            acs.at[m].set(traj["actions"])
            times.at[m].set(jnp.cumsum(traj["dts"]))
        loss = mb_agent.batched_update(i=i, obs=obs, acs=acs, times=times)
# well, on cpu and gpu it all takes around 16 seconds per iteration

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:44<00:00, 22.44s/it]


In [9]:
sample_n_trajectories(
    env=env,
    policy=mb_agent,
    ntraj=10,
    max_length=200,
    key=key
)

100%|██████████| 10/10 [00:53<00:00,  5.33s/it]


([{'observation': array([[-4.29383874e-01, -9.03122067e-01,  8.79941702e-01],
          [-4.09075439e-01, -9.12500560e-01,  9.23515037e-02],
          [-4.24911082e-01, -9.05235112e-01, -7.08403707e-01],
          [-4.75448489e-01, -8.79743576e-01, -1.47620964e+00],
          [-5.53506613e-01, -8.32844794e-01, -2.09960175e+00],
          [-6.52776182e-01, -7.57550836e-01, -2.80749822e+00],
          [-7.64784157e-01, -6.44286633e-01, -3.48910546e+00],
          [-8.70053172e-01, -4.92957920e-01, -3.84131694e+00],
          [-9.50821519e-01, -3.09739351e-01, -4.12997770e+00],
          [-9.95069563e-01, -9.91791785e-02, -4.43647861e+00],
          [-9.92724776e-01,  1.20405696e-01, -4.34687233e+00],
          [-9.45598245e-01,  3.25336754e-01, -4.08094358e+00],
          [-8.61557424e-01,  5.07660091e-01, -3.95307875e+00],
          [-7.53975332e-01,  6.56902730e-01, -3.44816518e+00],
          [-6.39233410e-01,  7.69012809e-01, -3.00566792e+00],
          [-5.28103769e-01,  8.49179864e